MLFLOW_TRACKING_URI=https://dagshub.com/dinesh-gaire/Kidney-Disease-Classification-Deep-Learning-Project.mlflow \
MLFLOW_TRACKING_USERNAME=dinesh-gaire \
MLFLOW_TRACKING_PASSWORD=128d72b7d0639152d7b82e850f4ab336e09cd862 \
python script.py

In [8]:
import os

In [9]:
%pwd

'/workspaces/Kidney-Disease-Classification-Deep-Learning-Project'

In [10]:
os.chdir("../")

In [11]:
%pwd

'/workspaces'

In [12]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/dinesh-gaire/Kidney-Disease-Classification-Deep-Learning-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="dinesh-gaire"
os.environ["MLFLOW_TRACKING_PASSWORD"]="128d72b7d0639152d7b82e850f4ab336e09cd862"

In [13]:
import tensorflow as tf

In [14]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

OSError: No file or directory found at artifacts\training\model.h5